In [83]:
import pandas as pd 
import json
import gzip
import pyarrow as pa
import pyarrow.parquet as pq

Carga de los datos


In [84]:
# Leemos el archivo json y lo guardamos en una lista
row = [] # creamos una lista vacia para ir agregando las filas del archivo json
with open ('datasets_json\output_steam_games.json') as file: # abrimos el archivo json
    for line in file.readlines(): # leemos linea por linea
        data = json.loads(line) #guardamos los datos de cada linea en una variable data 
        row.append(data)   #agregamos esa data a la lista vacia

# Convertimos la lista en un dataframe 
steam_games_original = pd.DataFrame(row)
steam_games_original.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Vemos como esta conformado el Dataframe

In [85]:
steam_games_original.shape #nos damos una idea de que tan grande es el Dataframe

(120445, 13)

In [86]:
steam_games_original.info() #Solicitamos mas informacion

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24083 non-null  object
 1   genres        28852 non-null  object
 2   app_name      32133 non-null  object
 3   title         30085 non-null  object
 4   url           32135 non-null  object
 5   release_date  30068 non-null  object
 6   tags          31972 non-null  object
 7   reviews_url   32133 non-null  object
 8   specs         31465 non-null  object
 9   price         30758 non-null  object
 10  early_access  32135 non-null  object
 11  id            32133 non-null  object
 12  developer     28836 non-null  object
dtypes: object(13)
memory usage: 11.9+ MB


In [87]:
steam_games_original.describe() #obtenemos informacion de cuales son las columnas con valores numericos 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
count,24083,28852,32133,30085,32135,30068,31972,32133,31465,30758.00,32135,32133,28836
unique,8239,883,32094,30054,32135,3582,15395,32132,4649,162.00,2,32132,10992
top,Ubisoft,[Action],Soundtrack,Soundtrack,http://store.steampowered.com/app/761140/Lost_...,2012-10-16,"[Casual, Simulation]",http://steamcommunity.com/app/612880/reviews/?...,[Single-player],4.99,False,612880,Ubisoft - San Francisco
freq,385,1880,3,3,1,100,1292,2,2794,4278.00,30188,2,1259


In [88]:
steam_games_original.isnull().sum() #vemos cuantos null hay dentro del dataframe

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

Proseguimos a ver los duplicados dentro del dataframe

In [89]:
cant_duplicados = steam_games_original[steam_games_original.duplicated(subset=['id'])] #como se puede observar a continuacion, hay 88312 filas x 13 columnas que estan llenas de datos tipo NaN 
cant_duplicados

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88384,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games


In [90]:
steam_games = steam_games_original.dropna(how="all").reset_index(drop=True) # lo que hacemos en esta linea, borramos todas las filas que contengan datos nulos en todas sus columnas
                                                                            # y por consiguiente restablecemos los indices del dataframe.
                                                                            # guardamos el dataframe sin nulos en una nueva variable.

In [91]:
steam_games.shape #como podemos observar, el nuevo dataframe sin nulos es mucho mas chico, ya que se borraron 88312 filas de datos nulos.

(32135, 13)

In [92]:
mas_duplicados = steam_games[steam_games.duplicated(subset=['id'])] # una vez que sacamos los valores nulos, nos concentramos en sacar filas que esten duplicadas.
mas_duplicados                                                      #como se puede ver, la fila 14573 y la 30961 son duplicadas, por lo cual procedemos a eliminarlas.

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


In [93]:
steam_games = steam_games.drop([14573, 30961]) # Eliminamos las filas 14573 y 30961 en especifico.

In [94]:
mas_duplicados = steam_games[steam_games.duplicated(subset=['id'])] # no quedan duplicados dentro del dataframe. 
mas_duplicados


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer


In [95]:
steam_games.drop(columns='title', inplace=True) #eliminamos la columna 'title' ya que contiene los mismos valores que la columna 'app_name'


In [96]:
steam_games.drop(columns='early_access', inplace=True) # eliminamos la columna 'early_access' ya que no sera util para los modelos propuestos.

In [97]:
steam_games.drop(columns='url', inplace=True) # eliminamos la columna 'url' ya que no la considero necesaria

In [98]:
steam_games

,publisher,genres,app_name,release_date,tags,reviews_url,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,767400,彼岸领域
4,NaN,NaN,Log Challenge,NaN,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN
...,...,...,...,...,...,...,...,...,...,...
32130,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,773640,"Nikita ""Ghost_RUS"""
32131,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,733530,Sacada
32132,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,658870,"xropi,stev3ns"


Transformacion de los datos

In [99]:
steam_games.dtypes # Antes que nada, tenemos que ver que tipo de datos tenemos.
                   # tal cual podemos observar, todos los datos son de tipo 'object', esto se debe a que los datos provienen de un archivo json
                   # el proximo paso, es desanidar estos datos, ya que lo mas seguro es que contengan un formato del tipo key-value. 

publisher       object
genres          object
app_name        object
release_date    object
tags            object
reviews_url     object
specs           object
price           object
id              object
developer       object
dtype: object

In [100]:
tipo_data = {"columna":[],"tipos_de_datos":[]} # creamos un diccionario vacio

for columna in steam_games.columns: # iteramos por las columnas del dataframe 
    tipo_data["columna"].append(columna) # cada columna dentro del df, se llamara columna.
    tipo_data["tipos_de_datos"].append(steam_games[columna].apply(type).unique()) # el dato que contenga cada columna, se agrega como tipos_de_datos

analisis_dtype= pd.DataFrame(tipo_data) # convertimos el diccionario en un dataframe para 
analisis_dtype

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'float'>]"
1,genres,"[<class 'list'>, <class 'float'>]"
2,app_name,"[<class 'str'>, <class 'float'>]"
3,release_date,"[<class 'str'>, <class 'float'>]"
4,tags,"[<class 'list'>, <class 'float'>]"
5,reviews_url,"[<class 'str'>, <class 'float'>]"
6,specs,"[<class 'list'>, <class 'float'>]"
7,price,"[<class 'float'>, <class 'str'>]"
8,id,"[<class 'str'>, <class 'float'>]"
9,developer,"[<class 'str'>, <class 'float'>]"


In [101]:
#luego de ver el analisis_dtype, saco la conclusion que seria mejor cambiar el tipo_de_datos de varias columnas. 
#En principio, empezaremos con la columna 'id'. notese que primero hay que eliminar las filas con valores nulos, de lo contrario no lo podremos hacer.

steam_games = steam_games.drop(steam_games.loc[steam_games['id'].isna()].index) # Eliminamos datos nulos de las filas, los cuales no tienen relevancia.

steam_games['id'] = steam_games['id'].astype(str) # Realizamos una conversion en el tipo_de_dato de la columna 'id' a str.

#renombramos 'id' por 'item_id'
steam_games = steam_games.rename(columns={'id': 'item_id'})


In [102]:
# Proseguimos con la columna 'release_date', la cual tiene una notacion 'YYYY-MM-DD' como podemos ver a continuacion
steam_games['release_date']

0        2018-01-04
1        2018-01-04
2        2017-07-24
3        2017-12-07
4               NaN
            ...    
32130    2018-01-04
32131    2018-01-04
32132    2018-01-04
32133    2017-09-02
32134           NaN
Name: release_date, Length: 32132, dtype: object

In [103]:
steam_games['release_date'].isnull().sum()

2066

In [104]:
#convertimos esta columna a datetime
steam_games['release_date'] = pd.to_datetime(steam_games['release_date'], errors='coerce')

In [105]:
# Calculamos la fecha media, con el proposito de reemplazar los nulos con la media.
mean_date = steam_games['release_date'].mean()
#ahora reemplazamos los nulos por la media
steam_games['release_date'].fillna(mean_date, inplace=True)

In [106]:
steam_games['release_date'].isnull().sum() #Comprobamos que efectivamente ya no hay nulos.

0

In [107]:
#Ahora preseguimos a tomar el año de la columna 'release_date' para hacer una nueva columna 'release_year'
steam_games['release_year'] = steam_games['release_date'].dt.year
steam_games.head()

,publisher,genres,app_name,release_date,tags,reviews_url,specs,price,item_id,developer,release_year
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04 00:00:00.000000000,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04 00:00:00.000000000,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24 00:00:00.000000000,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07 00:00:00.000000000,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,2015-04-21 10:04:45.040797952,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN,2015


In [108]:
#Como ya tenemos la columna 'release_year', procedemos a eliminar la columna 'release_date'
steam_games = steam_games.drop('release_date', axis=1)

In [109]:
#El proximo paso es sobre la columna 'genres' y 'tags', ya que ambas columnas comparten algunos datos. 
#Vamos a completar los valores faltantes de la columna 'tags' con los valores de la columna 'genres'
filas_vacias = steam_games['tags'].isna() #Guardamos las filas con valores NaN en tags
steam_games.loc[filas_vacias, 'tags'] = steam_games.loc[filas_vacias, 'genres'] # Asignamos los valores de la columna genres dentro de la columna tags

In [110]:
#Proseguimos con la columna 'genres', vamos a tomar un campo al azar para ver como estan distribuidos los datos.
steam_games['genres'][5]

['Action', 'Adventure', 'Simulation']

In [111]:
#vemos cuantos nulos hay dentro de la columna 'genre'
steam_games['genres'].isnull().sum()

3282

In [112]:
steam_games['genres'].value_counts() #vemos la cantidad de valores para cada genero

genres
[Action]                                                          1879
[Action, Indie]                                                   1650
[Simulation]                                                      1396
[Casual, Simulation]                                              1359
[Action, Adventure, Indie]                                        1082
                                                                  ... 
[Action, Adventure, Racing, Simulation, Strategy]                    1
[Action, Adventure, Casual, Indie, Racing, Sports, Strategy]         1
[Action, Adventure, Casual, Indie, Racing, Simulation, Sports]       1
[Action, Massively Multiplayer, RPG, Strategy]                       1
[Adventure, Casual, RPG, Simulation, Early Access]                   1
Name: count, Length: 883, dtype: int64

In [113]:
#como podemos observar, en la columna 'genres' hay varios generos que estan anidados en una lista.
#Vamos a hacer una funcion con la finalidad de limpiar esta columna y rellenar estas listas solo con el primer valor
#En el caso de los nulos, los clasificaremos como 'pending classification', para luego trabajarlos.
def clean_genres(genres):
    if isinstance(genres, list):
        return genres[0]
    elif isinstance(genres, str):
        return genres
    else:
        return 'Pending classification'

In [114]:
#usamos la funcion en la columna 'genres' y crearemos una nueva columna que contenga los datos de forma mas ordenada
steam_games['clean_genres'] = steam_games['genres'].apply(clean_genres)

In [115]:
#verificamos como quedaron los generos
steam_games['clean_genres'].value_counts()

clean_genres
Action                       11316
Casual                        4340
Adventure                     4328
Indie                         3299
Pending classification        3282
Simulation                    1852
Strategy                      1143
RPG                            685
Free to Play                   523
Racing                         454
Design &amp; Illustration      327
Animation &amp; Modeling       179
Utilities                       93
Audio Production                84
Sports                          63
Massively Multiplayer           49
Education                       46
Video Production                21
Software Training               21
Web Publishing                   8
Photo Editing                    8
Accounting                       7
Early Access                     4
Name: count, dtype: int64

 Como podemos ver, Pending classification son los valores nulos y esta entre los primeros generos con mas datos. Para evitar borrar esos datos que representan un valor considerable, implementaremos una clase que contenga funciones para los generos mas representativos. Dichas funciones recorreran la columna 'clean_genres' y en caso de encontrar un valor = 'pending classification', se reemplazara por un genero que corresponda de la columna 'tags'

In [116]:
class GenreClassifier:
    def replace_pending_with_action(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Action' in row['tags']:
                    return 'Action'
        return row['clean_genres']

    def replace_pending_with_indie(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Indie' in row['tags']:
                    return 'Indie'
        return row['clean_genres']

    def replace_pending_with_casual(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Casual' in row['tags']:
                    return 'Casual'
        return row['clean_genres']

    def replace_pending_with_adventure(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Adventure' in row['tags']:
                    return 'Adventure'
        return row['clean_genres']

    def replace_pending_with_strategy(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Strategy' in row['tags']:
                    return 'Strategy'
        return row['clean_genres']

    def replace_pending_with_rpg(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'RPG' in row['tags']:
                    return 'RPG'
        return row['clean_genres']

    def replace_pending_with_simulation(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Simulation' in row['tags']:
                    return 'Simulation'
        return row['clean_genres']

    def replace_pending_with_free_to_play(self, row):
        if row['clean_genres'] == 'Pending classification':
            if isinstance(row['tags'], list):
                if 'Free to Play' in row['tags']:
                    return 'Free to Play'
        return row['clean_genres']

In [117]:
# Creamos una instancia de la clase
genre_classifier = GenreClassifier()

In [118]:
#Aplicamos las funciones a la columna 'clean_genres'
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_action, axis=1)

In [119]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_indie, axis=1)

In [120]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_casual, axis=1)

In [121]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_adventure, axis=1)

In [122]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_strategy, axis=1)

In [123]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_rpg, axis=1)

In [124]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_simulation, axis=1)

In [125]:
steam_games['clean_genres'] = steam_games.apply(genre_classifier.replace_pending_with_free_to_play, axis=1)

In [126]:
#Una vez implementadas las funciones, volvemos a ver los valores de la columna 'clean_genres'
steam_games['clean_genres'].value_counts()

clean_genres
Action                       12428
Casual                        4528
Adventure                     4386
Indie                         3906
Simulation                    1927
Pending classification        1206
Strategy                      1152
RPG                            693
Free to Play                   542
Racing                         454
Design &amp; Illustration      327
Animation &amp; Modeling       179
Utilities                       93
Audio Production                84
Sports                          63
Massively Multiplayer           49
Education                       46
Video Production                21
Software Training               21
Web Publishing                   8
Photo Editing                    8
Accounting                       7
Early Access                     4
Name: count, dtype: int64

In [127]:
#Ya que la nueva columna 'clean_genres' es una transformacion de la columna original 'genres', borramos esta ultima.
steam_games = steam_games.drop('genres', axis=1)

In [128]:
#para que no haya confucion, cambiamos el nombre de la columna 'clean_genres' por 'genres'
steam_games = steam_games.rename(columns={'clean_genres': 'genres'})

In [129]:
steam_games['genres'].isnull().sum()

0

In [130]:
#Una vez modificada la columna 'genres', procedemos a eliminar la columna 'tags'
steam_games = steam_games.drop('tags', axis=1)

In [131]:
#Verificamos como queda el dataframe 
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     24081 non-null  object
 1   app_name      32131 non-null  object
 2   reviews_url   32132 non-null  object
 3   specs         31463 non-null  object
 4   price         30755 non-null  object
 5   item_id       32132 non-null  object
 6   developer     28834 non-null  object
 7   release_year  32132 non-null  int32 
 8   genres        32132 non-null  object
dtypes: int32(1), object(8)
memory usage: 3.3+ MB


In [132]:
#Ahora proseguimos con la columna 'price', ya que hay valores str. Haremos una funcion para convertir estos valores str en float 0.0
def convertir_price(valor):

    if pd.isna(valor): # si el valor es nulo retorno 0.0
        return 0.0
    try:
        flotante = float(valor) # si el valor es un flotante retorno el mismo valor
        return flotante
    except (ValueError, TypeError): # si el valor no es un flotante retorno 0.0
        return 0.0

In [133]:
#Reemplazamos todos los valores que no pueden convertirse a float, por 0.0, con la funcion convertir_price.
steam_games['price'] = steam_games['price'].apply(convertir_price) 
#Ahora los valores str tales como "Free to Play", se covirtieron en 0.0

In [134]:
steam_games['price'] #Aca podemos observar que la columna 'price'

0        4.99
1        0.00
2        0.00
3        0.99
4        2.99
         ... 
32130    1.99
32131    4.99
32132    1.99
32133    4.99
32134    4.99
Name: price, Length: 32132, dtype: float64

In [135]:
#Como proximo paso, vamos a ver cual es el porcentaje de nulos que quedan en cada columna
total_filas = len(steam_games)
nulos_x_columna = steam_games.isnull().sum()
porcentaje_nulos_x_columna = (nulos_x_columna / total_filas) * 100
porcentaje_nulos_x_columna

publisher       25.056019
app_name         0.003112
reviews_url      0.000000
specs            2.082037
price            0.000000
item_id          0.000000
developer       10.263911
release_year     0.000000
genres           0.000000
dtype: float64

In [136]:
steam_games['developer'].isnull().sum()

3298

Con respecto a los datos que vimos anteriormente, probamos con hacer una funcion que rellene los valores nulos de la columna 'developer' con los valores no nulos de la columna 'publisher'. Una vez hecha esta prueba, solo 30 y algun dato fueron reemplazados de los 3233. Teniendo en cuenta estos datos, decidimos dejar estos valores nulos tal como estan, y eliminar la columna 'publisher' ya que esta no es necesaria para los endpoints.

In [137]:
steam_games.head()

,publisher,app_name,reviews_url,specs,price,item_id,developer,release_year,genres
0,Kotoshiro,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,761140,Kotoshiro,2018,Action
1,"Making Fun, Inc.",Ironbound,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL,2018,Free to Play
2,Poolians.com,Real Pool 3D - Poolians,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com,2017,Casual
3,彼岸领域,弹炸人2222,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,767400,彼岸领域,2017,Action
4,NaN,Log Challenge,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,773570,NaN,2015,Action


In [138]:
#Proseguimos con la columna 'specs', ya que esta columna no contiene informacion relevante, la eliminamos
steam_games = steam_games.drop('specs', axis=1)

In [139]:
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32132 entries, 0 to 32134
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24081 non-null  object 
 1   app_name      32131 non-null  object 
 2   reviews_url   32132 non-null  object 
 3   price         32132 non-null  float64
 4   item_id       32132 non-null  object 
 5   developer     28834 non-null  object 
 6   release_year  32132 non-null  int32  
 7   genres        32132 non-null  object 
dtypes: float64(1), int32(1), object(6)
memory usage: 3.1+ MB


Teniendo en cuenta que la unica columna que queda con valores nulos es 'developer', decidimos eliminar dichos nulos para que no provoquen un sesgo en nuestro analisis, ya que esta columna es de mucho importancia para los endpoints.

In [140]:
#filtramos y mantenemos las columnas donde no hayan nulos
steam_games = steam_games[steam_games['developer'].notnull()]

#Reseteamos los indices despues de filtrar
steam_games = steam_games.reset_index(drop=True)

In [141]:
steam_games['developer'].isnull().sum()

0

In [142]:
steam_games.head() #como podemos observar, el dataframe ya quedo como queremos. sin nulos y con una disposicion homogenea.

,publisher,app_name,reviews_url,price,item_id,developer,release_year,genres
0,Kotoshiro,Lost Summoner Kitty,http://steamcommunity.com/app/761140/reviews/?...,4.99,761140,Kotoshiro,2018,Action
1,"Making Fun, Inc.",Ironbound,http://steamcommunity.com/app/643980/reviews/?...,0.00,643980,Secret Level SRL,2018,Free to Play
2,Poolians.com,Real Pool 3D - Poolians,http://steamcommunity.com/app/670290/reviews/?...,0.00,670290,Poolians.com,2017,Casual
3,彼岸领域,弹炸人2222,http://steamcommunity.com/app/767400/reviews/?...,0.99,767400,彼岸领域,2017,Action
4,Trickjump Games Ltd,Battle Royale Trainer,http://steamcommunity.com/app/772540/reviews/?...,3.99,772540,Trickjump Games Ltd,2018,Action


In [143]:
analisis_dtype

,columna,tipos_de_datos
0,publisher,"[<class 'str'>, <class 'float'>]"
1,genres,"[<class 'list'>, <class 'float'>]"
2,app_name,"[<class 'str'>, <class 'float'>]"
3,release_date,"[<class 'str'>, <class 'float'>]"
4,tags,"[<class 'list'>, <class 'float'>]"
5,reviews_url,"[<class 'str'>, <class 'float'>]"
6,specs,"[<class 'list'>, <class 'float'>]"
7,price,"[<class 'float'>, <class 'str'>]"
8,id,"[<class 'str'>, <class 'float'>]"
9,developer,"[<class 'str'>, <class 'float'>]"


In [145]:
# Lista de columnas para convertir a tipo string
columnas_a_convertir = ['app_name', 'reviews_url']

# Aplicar la conversión a tipo string
steam_games[columnas_a_convertir] = steam_games[columnas_a_convertir].astype(str)


PROCEDEMOS A GUARDAR EL DATAFRAME COMO CSV


In [146]:
steam_games.to_csv('dataset_limpio/steam_games.csv', index=False, encoding='UTF-8') #convertimos el dataframe a csv y ponemos index=False para que no haga una nueva columna de indice.

CONVERTIMOS EL ARCHIVO CSV A PARQUET


In [147]:
steam_games = pd.read_csv("dataset_limpio/steam_games.csv") # leemos el archivo csv

tabla = pa.Table.from_pandas(steam_games) # convertimos el dataframe en una tabla
pq.write_table(tabla,"dataset_limpio/steam_games.parquet") # guardamos la tabla en un archivo parquet